<a href="https://colab.research.google.com/github/matehr/stockmarket/blob/master/StockResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install edgar
import json
import requests
import edgar

In [5]:
ticker = input("What stock ticker would you like to research?")

response = requests.get("https://financialmodelingprep.com/api/v3/financials/income-statement/" + ticker)

print(response.json())

What stock ticker would you like to research?CCL
{'symbol': 'CCL', 'financials': [{'date': '2018-11-30', 'Revenue': '18881000000.0', 'Revenue Growth': '0.0782981153626499', 'Cost of Revenue': '11089000000.0', 'Gross Profit': '7792000000.0', 'R&D Expenses': '0.0', 'SG&A Expense': '2450000000.0', 'Operating Expenses': '4467000000.0', 'Operating Income': '3325000000.0', 'Interest Expense': '194000000.0', 'Earnings before Tax': '3206000000.0', 'Income Tax Expense': '54000000.0', 'Net Income - Non-Controlling int': '0.0', 'Net Income - Discontinued ops': '0.0', 'Net Income': '3152000000.0', 'Preferred Dividends': '0.0', 'Net Income Com': '3152000000.0', 'EPS': '4.45', 'EPS Diluted': '4.44', 'Weighted Average Shs Out': '725174680.0', 'Weighted Average Shs Out (Dil)': '725174680.0', 'Dividend per Share': '1.95', 'Gross Margin': '0.4127', 'EBITDA Margin': '0.287', 'EBIT Margin': '0.1801', 'Profit Margin': '0.167', 'Free Cash Flow margin': '0.1159', 'EBITDA': '5417000000.0', 'EBIT': '3400000000